In [56]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

from sklearn.preprocessing import QuantileTransformer
from joblib import load

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model.joblib")


In [57]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [58]:
df = pd.read_sql(
    """
    select ticker, roeq, bm, mom12m, mve
    from today
    """, 
    conn
)
df = df.dropna()
df = df.set_index("ticker")


In [59]:
df["size_rnk"] = df.mve.rank(ascending=False)
df = df[df.size_rnk>500]

In [60]:
features = ["roeq", "bm", "mom12m"]
df[features] = qt.fit_transform (df[features])
df["predict"] = pipe.predict(df[features])
df.to_csv("files/predictions.csv")

In [61]:
df.head()

,roeq,bm,mom12m,mve,size_rnk,predict
ticker,,,,,,
AADI,-1.353305,-0.372963,-1.006508,5.739793,1979.0,-0.102062
AAN,-0.642654,1.419806,-1.060369,6.057252,1851.0,-0.043816
AAON,0.931040,-1.174550,0.853046,8.295798,759.0,0.048124
AAT,-0.236481,0.249704,-0.403236,7.417280,1229.0,0.020684
AAWW,0.003878,1.571010,0.962537,7.968423,943.0,0.059704


In [62]:
import plotly.graph_objects as go 
import numpy as np

cd = np.empty(shape=(df.shape[0], 3, 1), dtype=float)
cd[:, 0] = df.roeq.to_numpy().reshape(-1, 1)
cd[:, 1] = df.bm.to_numpy().reshape(-1, 1)
cd[:, 2] = df.mom12m.to_numpy().reshape(-1, 1)
string = """
    %{text}<br>
    roeq = %{customdata[0]:.2}<br>
    bm = %{customdata[1]:.2}<br>
    mom12m = %{customdata[2]:.2}<extra></extra>
    """

trace = go.Scatter(
    x=df.roeq, 
    y=df.predict, 
    mode="markers",
    text=df.index.to_list(),
    customdata=cd,
    hovertemplate=string,
    marker=dict(
        size=10,
        opacity=0.5
    )
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="roeq",
    xaxis_title_font_size=16,
    xaxis_showgrid=True,
    yaxis_title_font_size=16,
    yaxis_showgrid=True,
    yaxis_title="prediction",
)
fig.show()

In [63]:
trace = go.Scatter(
    x=df.bm, 
    y=df.predict, 
    mode="markers",
    text=df.index.to_list(),
    customdata=cd,
    hovertemplate=string,
    marker=dict(
        size=10,
        opacity=0.5
    )
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="bm",
    xaxis_title_font_size=16,
    xaxis_showgrid=True,
    yaxis_title_font_size=16,
    yaxis_showgrid=True,
    yaxis_title="prediction",
)
fig.show()

In [64]:
trace = go.Scatter(
    x=df.mom12m, 
    y=df.predict, 
    mode="markers",
    text=df.index.to_list(),
    customdata=cd,
    hovertemplate=string,
    marker=dict(
        size=10,
        opacity=0.5
    )
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="mom12m",
    xaxis_title_font_size=16,
    xaxis_showgrid=True,
    yaxis_title_font_size=16,
    yaxis_showgrid=True,
    yaxis_title="prediction",
)
fig.show()

In [65]:
df = df.sort_values(by="predict")
df2 = pd.concat((df.iloc[:200], df.iloc[-200:]))

cd = np.empty(shape=(df2.shape[0], 3, 1), dtype=float)
cd[:, 0] = df2.roeq.to_numpy().reshape(-1, 1)
cd[:, 1] = df2.bm.to_numpy().reshape(-1, 1)
cd[:, 2] = df2.mom12m.to_numpy().reshape(-1, 1)
string = """
    %{text}<br>
    roeq = %{customdata[0]:.2}<br>
    bm = %{customdata[1]:.2}<br>
    mom12m = %{customdata[2]:.2}<extra></extra>
    """

trace = go.Scatter(
    x=df2.roeq, 
    y=df2.predict, 
    mode="markers",
    text=df2.index.to_list(),
    customdata=cd,
    hovertemplate=string,
    marker=dict(
        size=10,
        opacity=0.5
    )
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="roeq",
    xaxis_title_font_size=16,
    xaxis_showgrid=True,
    yaxis_title_font_size=16,
    yaxis_showgrid=True,
    yaxis_title="prediction",
)

In [66]:
trace = go.Scatter(
    x=df2.bm, 
    y=df2.predict, 
    mode="markers",
    text=df2.index.to_list(),
    customdata=cd,
    hovertemplate=string,
    marker=dict(
        size=10,
        opacity=0.5
    )
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="bm",
    xaxis_title_font_size=16,
    xaxis_showgrid=True,
    yaxis_title_font_size=16,
    yaxis_showgrid=True,
    yaxis_title="prediction",
)

In [67]:
trace = go.Scatter(
    x=df2.mom12m, 
    y=df2.predict, 
    mode="markers",
    text=df2.index.to_list(),
    customdata=cd,
    hovertemplate=string,
    marker=dict(
        size=10,
        opacity=0.5
    )
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="mom12m",
    xaxis_title_font_size=16,
    xaxis_showgrid=True,
    yaxis_title_font_size=16,
    yaxis_showgrid=True,
    yaxis_title="prediction",
)